# Project 3: Understanding data representation through low-dimensional projections (subspace learning)

### Name: Steve Nathan de Sa
### Course Level: CSC 448

### Due: Monday March 24, 2025

**Introduction:**
* In this project, we explore the application of classification using: a) Principal Componenet Analysis (PCA) and b) Linear Discriminant Analysis (LDA).

**Objectives:**
* The objective of this project is to implement different classification models to analyze real-world datasets, understand the relationship between variables, and perform classifications.  Additionally, students will gain experience understanding optimization techniques, linear algebra, and subspace learning.

## All Students

* The first problem we aim to design a pose (orientation) estimator for a set of image data.  Please download the Boat.zip file from the D2L project 3 module.

In [ ]:
import os
import struct
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.image import imread

**Problem A (60pts)**

1 (5pts). The first thing you'll need to do is read the image data in and construc the image data matrix $X = [\textbf{x}_1, \textbf{x}_2, \cdots, \textbf{x}_n]$ where each column of $X$ is a "row-scanned" image
$$
    \textbf{x}_i = \text{vec}(I_i) \in \mathbb{R}^{m}
$$
where $m$ is the number of pixels in the image (i.e., turn the image into a single column vector - hint: np.reshape works here)

For this particular dataset, there are $n=128$ images sampled at a different orientation around a single degree of freedom (i.e, the object being rotated around a single axis through 360 degrees).  Each image $I \in [0,1]^{h \times v}$ where $h=v=128$ and $m=16,384$ features.  

* This will result in a image data matrix $X \in \mathbb{R}^{16,384 \times 128}$

**Note:** The .zip also contains the Testing Set (same object imaged at random angles around the same single degree of freedom) along with a .txt file providing the actual pose (orientation) of each training image.

In [ ]:
# Read in the data (however you prefer), and Construct the Image data matrix X #
train_path = "Boat/Train/UnProcessed"
test_path = "Boat/Test/Boat32/UnProcessed"
pose_path = "Boat/Test"
pose_filename = 'RandAng.txt'

# Load training data
def load_training_data(train_path):
    n_images = 128
    img_size = 128 * 128
    X_train = np.zeros((img_size, n_images))
    
    for i in range(n_images):
        img_path = os.path.join(train_path, f'img_{i}.png')
        img = imread(img_path)
        X_train[:, i] = img.reshape(-1)
    
    return X_train

# Load test data
def load_test_data(test_path):
    n_test = 64
    img_size = 128 * 128
    X_test = np.zeros((img_size, n_test))
    
    for i in range(n_test):
        img_path = os.path.join(test_path, f'img_{i}.png')
        img = imread(img_path)
        X_test[:, i] = img.reshape(-1)
    
    return X_test

# Load + Read true poses
def read_true_poses(pose_path, pose_filename):
    pose_file_path = os.path.join(pose_path, pose_filename)
    
    with open(pose_file_path, 'r') as f:
        content = f.read().strip()
        poses_str = content.split(',')
        true_poses = [float(pose) for pose in poses_str]
    
    return np.array(true_poses)

# Load matrix data [X]
X_train = load_training_data(train_path)
X_test = load_test_data(test_path)

# Set Info Recovery ratio
mu_set = 0.9

2 (5pts). Next, we need to ensure the data is centered, so we subtract the mean image vector $\bar{\textbf{x}}$ from each image in the dataset.  Keep this for later classification! Write a function called CenterData that takes the original $X$ and returns the centered $X$ (referred to as $\bar{X}$) along with the mean vector $\bar{\textbf{x}}$ associated with $X$.

In [ ]:
# Returns the classification rate between models predicted target and true value #
def CenterData(d_X):
    mean_x = np.mean(d_X, axis=1, keepdims=True)
    barX = d_X - mean_x
    return barX, mean_x

3 (10pts). We will need a way to "select" the optimum subspace dimension (i.e., how many principal components to keep).  One way to do this is to use the information recovery ratio defined as:
$$
    \rho(\bar{X},\mu) = \frac{\sum_{i=1}^k \sigma_i^2}{\| \bar{X} \|^2_F} \leq \mu
$$
where $\mu \in [0,1]$ is the amount of information recovered from a $k$-dimensional subspace (note $\rho \rightarrow 1$ as $k \rightarrow n$) resulting in 100\% recovery and $\sigma_i$ is the $i^{\textbf{th}}$ singular value of $\bar{X}$.

* Write a function called InfoRecov that takes the \underline{centered} image data matrix $\bar{X}$, the mean vector $\bar{\textbf{x}}$, and user specified information recovery $\mu$ that returns the value $k$ required to acheive the user-specified information recovery.


**Note:** Generally $\mu=0.85 - 0.9$ is sufficient for good classification.

In [ ]:
# Information Recovery
def InfoRecov(barX, mean_x, mu):
    U, S, Vt = np.linalg.svd(barX, full_matrices=False) # SVD decomp
    
    # Calculate cumulative energy ratio
    sigma_squared = S**2
    frobenius_norm = np.sum(sigma_squared)
    cumulative_energy = np.cumsum(sigma_squared) / frobenius_norm
    
    # Find k that achieves desired information recovery ratio
    k_keep = np.argmax(cumulative_energy >= mu) + 1
    return k_keep

4 (20pts). Next write a function (called PCA) that takes the image data matrix $X$, and user specified accuracy $\mu$, to compute and return the first $k$ principal components of $X$ where $k$ is the subspace dimension required to acheive an information recovery $\mu$.  

**Note:** You will need to use the above "helper" functions to:

1. Ensure the data is centered prior to computation
2. Compute how many pricipal components to keep via the user specified information recovery $\mu$

In [ ]:
# Compute logistic regression model parameters #
def PCA(data_X, mu):
    barX, mean_x = CenterData(data_X) # Center data
    
    U, S, Vt = np.linalg.svd(barX, full_matrices=False) # Get SVD
    
    k_keep = InfoRecov(barX, mean_x, mu) # How many comps to keep?
    
    u_pca = U[:, :k_keep] # Get first k comps
    
    return u_pca

5 (10pts). Next write a function called DataProjection that takes the first $k$ principal components, the centered data matrix $\bar{X}$, and computes the change of basis $Y$ embedding the data into a $k$-dimensional subspace.  To illustrate your function is working, generate a $k=3$-dimensional plot showing the low-dimensional embedding in $\mathbb{R}^3$.

In [ ]:
# Data Projection
def DataProjection(barX, u_pca):
    y_embed = u_pca.T @ barX
    return y_embed

# Center data & plot k=3 for visualization
barX_train, mean_x = CenterData(X_train)
u_pca_3 = PCA(X_train, mu_set)[:, :3]
Y_3 = DataProjection(barX_train, u_pca_3)

# Just to make sure its k = 3
if u_pca_3.shape[1] == 3:
    fig = plt.figure(figsize=(10, 8))
    ax = fig.add_subplot(111, projection='3d')
    ax.scatter(Y_3[0, :], Y_3[1, :], Y_3[2, :])
    ax.set_xlabel('PC1')
    ax.set_ylabel('PC2')
    ax.set_zlabel('PC3')
    ax.set_title('3D Projection of Training Image Data')
    plt.show()

6 (10pts). Evaluate how well your low-dimensional projection works by estimating the pose of each image in the Test Set.  You may want to write a quick helper function here but it's not required.  The process for evaluation is as follows:

1. Vectorize a test image and subtract the mean computed above $\bar{\textbf{x}}$.
2. Project the test image onto the low-dimensional subspace (performa a change of basis $\textbf{t}=U_k^T \textbf{f}$) where $U_k \in \mathbb{R}^{m \times k}$ contains the first $k$ principal components, $\textbf{f} \in \mathbb{R}^m$ is the mean-centered vectorized image, and $\textbf{t} \in \mathbb{R}^k$ is the low-dimensional projection.
3. Perform a nearest-neighbor search with all other samples in $Y$ to compute which sample in $Y$ that $\textbf{t}$ is closest to.  
4. Use the equation $est = \frac{2 \pi i}{n}$ to estimate the orientation of the test sample (where $i$ is the closest sample to $\textbf{t}$)
5. Compute the estimation error by looking at the .txt file to determine the "true" orientation and subtracting the two.
6. Do this for all 64 test samples to determine the average classification error and standard deviation across the testing set.

In [ ]:
# Perform pose estimation and return the estimation results (mean and standard deviation) #

# Pose Estimation
def estimate_poses(X_train, X_test, test_path, pose_filename, mu):
    barX_train, mean_x = CenterData(X_train) # Center train data
    
    u_pca = PCA(X_train, mu) # Get principal comps
    
    Y_train = DataProjection(barX_train, u_pca) # Project to lower dim
    
    barX_test = X_test - mean_x # Center using mean
    
    T_test = u_pca.T @ barX_test # Project to lower dim
    
    # Read true poses
    true_poses = read_true_poses(pose_path, pose_filename)
    
    n_train = X_train.shape[1]
    errors = []
    
    # For each test image
    for i in range(X_test.shape[1]):
        distances = np.linalg.norm(Y_train - T_test[:, i:i+1], axis=0) # Calc dist to all train samples
        
        nearest_idx = np.argmin(distances) # Get nearest neighbor
        
        est_pose = 2 * np.pi * nearest_idx / n_train # Est pose using nearest neighbor's index
        
        true_pose = true_poses[i] # Get true pose
        
        # Make sure they are in range 0 -> 2π
        est_pose = est_pose % (2 * np.pi)
        true_pose = true_pose % (2 * np.pi)
        
        # Calc smallest circular difference
        error = min(abs(est_pose - true_pose), 2 * np.pi - abs(est_pose - true_pose))
        errors.append(error)
    
    errors = np.array(errors)    
    mean_error = np.mean(errors)
    std_error = np.std(errors)
    
    return mean_error, std_error, errors

# Run pose estimation
mean_err, std_err, all_errors = estimate_poses(X_train, X_test, test_path, pose_filename, mu_set)

# Print results
print(f"Mean estimation error (radians): {mean_err:.4f} (degrees): {np.degrees(mean_err):.2f}")
print(f"Std Dev of error (radians): {std_err:.4f} (degrees): {np.degrees(std_err):.2f}")

**Problem B (45pts)**


1 (5pts). Next, let's look at a multi-class problem and compute a linear subspace (change of basis) by exploring Linear Discriminant Analysis.  

Here, we want to investigate the first three odd digits of the MNIST dataset.  Let's go ahead and get the dataset loaded (60,000 training images and 10,000 testing images)

**The data can be downloaded** [Here](https://www.kaggle.com/datasets/hojjatk/mnist-dataset)

- Note that once loaded, you will need to "parse" the training and testing set to only keep the first three odd numbers (1,3, and 5).
- These are also images (size: $28 \times 28$) so you'll need to perform the same "vectorization" of each image into a 784-dimensional vector.


Let's also plot a few digits to get the hang of working with the dataset by creating a $3 \times 3$ subplot to display 9 total digits.

**Note:** When displaying images you need to provide a colormap, these are grayscale images so our colormap will be 'gray'

* $\texttt{plt.imshow(I,cmap='gray')}$ should work.

In [ ]:
# load MNIST data, separte the data into only 1,3 and 5s for both training and testing, plot the first 9 digits #
mnist_data_dir = 'archive'

# Special extrn fn for imgs
def load_idx_images(file_path):
    with open(file_path, 'rb') as f:
        magic, num, rows, cols = struct.unpack(">IIII", f.read(16))
        images = np.frombuffer(f.read(), dtype=np.uint8).reshape(num, rows * cols)
    return images

# Special extrn fn for labels
def load_idx_labels(file_path):
    with open(file_path, 'rb') as f:
        magic, num = struct.unpack(">II", f.read(8))
        labels = np.frombuffer(f.read(), dtype=np.uint8)
    return labels

# Load MNIST data
def load_mnist_data(data_dir):
    files = {
        'train_images': 'train-images.idx3-ubyte',
        'train_labels': 'train-labels.idx1-ubyte',
        'test_images': 't10k-images.idx3-ubyte',
        'test_labels': 't10k-labels.idx1-ubyte'
    }
    
    data = {
        'train_images': load_idx_images(os.path.join(data_dir, files['train_images'])),
        'train_labels': load_idx_labels(os.path.join(data_dir, files['train_labels'])),
        'test_images': load_idx_images(os.path.join(data_dir, files['test_images'])),
        'test_labels': load_idx_labels(os.path.join(data_dir, files['test_labels']))
    }
    
    return data

mnist_data = load_mnist_data(mnist_data_dir)

# Extract only 1, 3, 5
def extract_1_3_5_digits(images, labels, odd_digits=[1, 3, 5]):
    mask = np.isin(labels, odd_digits)
    return images[mask], labels[mask]

# Extract data
X_train, y_train = extract_1_3_5_digits(mnist_data['train_images'], mnist_data['train_labels'])
X_test, y_test = extract_1_3_5_digits(mnist_data['test_images'], mnist_data['test_labels'])

# Normalize data
X_train = X_train / 255.0
X_test = X_test / 255.0

# Add random noise as per Dr Hoover's email suggestion
X_train = X_train + 0.01 * np.random.rand(X_train.shape[0], X_train.shape[1])
X_test = X_test + 0.01 * np.random.rand(X_test.shape[0], X_test.shape[1])

# Plot 3x3 grid of first 9 digits
plt.figure(figsize=(8, 8))
for i in range(9):
    plt.subplot(3, 3, i+1)
    plt.imshow(X_train[i].reshape(28, 28), cmap='gray')
    plt.title(f"Digit: {y_train[i]}")
    plt.axis('off')
plt.tight_layout()
plt.show()

2 (30pts). Similar to the above problem A, write a function called LDA that takes the image data matrix (this can be sorted by class to make your life easier) and the number of classes and returns the $C-1$-dimensional change of basis that maximizes:
$$
    J(\textbf{w}) = \frac{\textbf{w}^T S_b \textbf{w}}{\textbf{w}^T S_w \textbf{w}}
$$
where $S_w$ is the $\textbf{within-class}$ scatter matrix and $S_b$ is the $\textbf{between-class}$ scatter matrix.

In [ ]:
# LDA Implementation
def LDA(x_data, y_data, c_classes):
    n_samples, n_features = x_data.shape
    unique_classes = np.unique(y_data)
    mean_overall = np.mean(x_data, axis=0).reshape(n_features, 1) # Overall mean
    
    S_b = np.zeros((n_features, n_features)) # Betn class scatter mat
    S_w = np.zeros((n_features, n_features)) # Within class scatter mat
    
    # Get class means and scatter matrices
    for c in unique_classes:
        X_c = x_data[y_data == c]
        n_c = X_c.shape[0]
        mean_c = np.mean(X_c, axis=0).reshape(n_features, 1) # Class mean
        
        # Betn class scatter
        mean_diff = mean_c - mean_overall
        S_b += n_c * np.dot(mean_diff, mean_diff.T)
        
        # Within class scatter
        for i in range(n_c):
            x_i = X_c[i].reshape(n_features, 1)
            x_diff = x_i - mean_c
            S_w += np.dot(x_diff, x_diff.T)

    # Better formula version: inv(S_w) * S_b * w = lambda * w (Using this one)
    
    epsilon = 1e-10 # Regularize S_w to avoid singularity issues
    S_w_reg = S_w + epsilon * np.eye(n_features)
    
    # Inverse of S_w
    try:
        S_w_inv = np.linalg.inv(S_w_reg)
    except np.linalg.LinAlgError:
        S_w_inv = np.linalg.pinv(S_w_reg) # Use pseudo-inverse instead
    
    # Compute M = inv(S_w) * S_b
    M = np.dot(S_w_inv, S_b)
    
    # Compute eigenvals + eigenvecs of M
    eigenvalues, eigenvectors = np.linalg.eig(M)
    
    # Sort eigenvals and corr eigenvecs (desc)
    idx = eigenvalues.argsort()[::-1]
    eigenvalues = eigenvalues[idx]
    eigenvectors = eigenvectors[:, idx]
    
    # Get top (c_classes - 1) eigenvecs
    w = eigenvectors[:, :c_classes-1].real
    
    return w

3 (10pts). To investigate the how well the LDA classifier works, let's use the testing set to determine the classification rates for:

* The entire testing set (assuming it has been pruned to digits 1,3 and 5)
* The classification rate for each individual digit (this will give you a feel for which representations are easier for the model to classify)

In [ ]:
# Evaluate Performance #
print("Wait! This may take a while :)")

# Perform LDA
n_classes = 3  # Corr to 1 3 5
lda_vectors = LDA(X_train, y_train, n_classes)

# Project data onto LDA subspace
X_train_lda = np.dot(X_train, lda_vectors)
X_test_lda = np.dot(X_test, lda_vectors)

# Classify using nearest centroid classifier
def nearest_centroid_classifier(X_train, y_train, X_test):
    # Want centroids for each class
    unique_classes = np.unique(y_train)
    centroids = np.zeros((len(unique_classes), X_train.shape[1]))
    
    for i, c in enumerate(unique_classes):
        centroids[i] = np.mean(X_train[y_train == c], axis=0)
    
    y_pred = np.zeros(X_test.shape[0], dtype=int) # Test data class pred
    
    for i in range(X_test.shape[0]):
        distances = np.sqrt(np.sum((centroids - X_test[i])**2, axis=1)) # Euc dist to centroids
        y_pred[i] = unique_classes[np.argmin(distances)] # Assign class
    
    return y_pred

# Perform classification on the LDA-projected test data
y_pred = nearest_centroid_classifier(X_train_lda, y_train, X_test_lda)

# Calculate overall accuracy
overall_accuracy = np.mean(y_pred == y_test) * 100
print(f"\nOverall Classification Accuracy: {overall_accuracy:.2f}%\n")

# Calculate accuracy per digit
unique_classes = np.unique(y_test)
for c in unique_classes:
    mask = (y_test == c)
    accuracy = np.mean(y_pred[mask] == y_test[mask]) * 100
    print(f"Digit {c} Classification Accuracy: {accuracy:.2f}%")

**Problem C (10pts)**
* In Problem B, you investigated classifying the first three odd digits in the MNIST dataset (digits 1,3, and 5).  Considering that LDA returns at most $C-1$ subspace vectors, we can easily plot these vectors (for this problem) in $\mathbb{R}^2$ to illustrate how well the class separation is.  Let's compare this plot with our PCA subspace to see which of the two methods gives better class separation (i.e., generate a subplot with the 2-dimensional PCA embedding on the left and the 2-dimensional LDA embedding on the right)



In [ ]:
# PCA implementation
def PCA(X, n_components):
    X_centered = X - np.mean(X, axis=0) # Center data
    cov_matrix = np.dot(X_centered.T, X_centered) / X.shape[0] # Convariance mat
    
    eigenvalues, eigenvectors = np.linalg.eigh(cov_matrix) # Get eigenvals + eigenvecs
    
    # Sort eigenvals and corr eigenvecs (desc)
    idx = eigenvalues.argsort()[::-1]
    eigenvalues = eigenvalues[idx]
    eigenvectors = eigenvectors[:, idx]
    
    # Select top n_components eigenvecs
    components = eigenvectors[:, :n_components]
    
    return components

# Perform PCA with 2 components
pca_vectors = PCA(X_train, 2)

# Project data onto PCA subspace
X_train_pca = np.dot(X_train, pca_vectors)
X_test_pca = np.dot(X_test, pca_vectors)

# LDA vs PCA projection
plt.figure(figsize=(14, 6))

# PCA projection
plt.subplot(1, 2, 1)
colors = ['r', 'g', 'b']
for i, c in enumerate(unique_classes):
    plt.scatter(X_train_pca[y_train == c, 0], X_train_pca[y_train == c, 1], 
                c=colors[i], label=f'Digit {c}', alpha=0.6)
plt.title('PCA Projection')
plt.xlabel('First PrincipalComponent')
plt.ylabel('Second PrincipalComponent')
plt.legend()
plt.grid(True)

# LDA projection
plt.subplot(1, 2, 2)
for i, c in enumerate(unique_classes):
    plt.scatter(X_train_lda[y_train == c, 0], X_train_lda[y_train == c, 1], 
                c=colors[i], label=f'Digit {c}', alpha=0.6)
plt.title('LDA Projection')
plt.xlabel('First Discriminant')
plt.ylabel('Second Discriminant')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

# LDA >>>>>> PCA